In [113]:
import pickle
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import os


# pkl 파일을 열고 객체를 불러오는 함수
def load_pickle(file_path):
    with open(file_path, 'rb') as f:  # 'rb'는 바이너리 읽기 모드
        data = pickle.load(f)  # pickle.load를 사용해 파일 내용 불러오기
    return data

# 예시: 'example.pkl' 파일을 불러오기
file_path = '/data/ephemeral/home/mmdetection/result_ddq_final_epoch7.pkl'
loaded_data = load_pickle(file_path)

In [114]:
loaded_data[0]['pred_instances'].keys()

dict_keys(['scores', 'labels', 'bboxes'])

In [115]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO('../dataset/test.json')
img_ids = coco.getImgIds()
epoch = 'ddq_tta'
work_dir = './work_dirs/ddq_inference/20241021_112244'

class_num = 10
for i, out in zip(img_ids,loaded_data):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    box_num = sum(out['pred_instances']['scores']>0.1)
    for o in range(box_num):
        score, label, box   = out['pred_instances'].values()
        prediction_string += str(label[o].item()) + ' ' + str(score[o].item()) + ' ' + str(box[o][0].item()) + ' ' + str(box[o][1].item()) + ' ' + str(box[o][2].item()) + ' ' + str(box[o][3].item()) + ' '
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,7 0.8813168406486511 118.19282531738281 424.30...,test/0000.jpg
1,4 0.8049694299697876 343.3912353515625 250.294...,test/0001.jpg
2,1 0.7723322510719299 301.8743591308594 317.855...,test/0002.jpg
3,9 0.7978998422622681 146.0308837890625 262.569...,test/0003.jpg
4,1 0.7177478075027466 194.99676513671875 253.65...,test/0004.jpg


In [99]:
import pandas as pd
import numpy as np

csv_file = pd.read_csv('./work_dirs/ddq_inference/20241023_215350/submission_ddq_epoch7_tta.csv')
csv_file['PredictionString']

0       7 0.7335732579231262 606.1556396484375 523.272...
1       4 0.7177553772926331 347.1719665527344 253.621...
2       1 0.46254557371139526 311.02032470703125 341.2...
3       9 0.8191002607345581 149.83074951171875 266.62...
4       1 0.6632032990455627 199.39907836914062 260.75...
                              ...                        
4866    0 0.7160687446594238 295.73150634765625 348.93...
4867    3 0.8295949101448059 417.6707763671875 320.705...
4868    7 0.49762263894081116 118.4051513671875 467.07...
4869    0 0.5557988286018372 25.858369827270508 252.21...
4870    4 0.8133053779602051 375.245849609375 208.5416...
Name: PredictionString, Length: 4871, dtype: object

In [95]:
def ensemble_threshold(csv_file,threshold=0.3):
    threshold_csv = []
    for n,i in enumerate(csv_file['PredictionString']):
        try:
            threshold_csv.append(' '.join(i.split()[:6*sum(np.array(list(map(float,i.split()[1::6])))>threshold)]))
        except:
            threshold_csv.append([])
    csv_file['PredictionString'] = threshold_csv
    return csv_file

tt = ensemble_threshold(csv_file,0.1)